In [1]:
import requests
import os
import json

import pickle as pkl

import TwitterUtils as TU

## Exploratory Space For Developing Twitter API Workflow
Our first step is to sample some arbitrary number of twitter users. We have done this in the TwitterUtils.py file and are using this document to develop the workflow.

In [2]:
user_seed = TU.TwitterClient()
rules = user_seed.get_rules() # Prints Rule for current Stream; Need to determine if we can repurpose the get_stream to another endpoint
user_sample = user_seed.get_stream(rules, sample_size = 100)

{"data": [{"id": "1633877176914149393", "value": "(a OR the) has:geo lang:en tweets_count:50", "tag": "active_user"}], "meta": {"sent": "2023-03-09T18:04:09.149Z", "result_count": 1}}
200


In [4]:
user_ids = [tweet['data']['author_id'] for tweet in user_sample]
user_ids

In [5]:
user_ids

['3871965339',
 '1168147256878477312',
 '1096491912100544513',
 '711275933211217920',
 '3030032723',
 '780584744941518848',
 '1208877273455181824',
 '4236586214',
 '1183482031809945601',
 '61642501',
 '1433318794248736771',
 '18503412',
 '14807763',
 '46634281',
 '9544202',
 '25928487',
 '2561986016',
 '1149492653291008000',
 '1507922421252341764',
 '1423573289763577859',
 '996250724',
 '357590792',
 '1573722308237266947',
 '255139039',
 '1104105195561238529',
 '1188165314174828544',
 '1571077994180972547',
 '2777754233',
 '255139039',
 '361879829',
 '128501292',
 '1094683520',
 '31551908',
 '1311829941584039936',
 '255139039',
 '23172795',
 '253036174',
 '43729216',
 '1653062239',
 '335713584',
 '1462902139462701061',
 '459300961',
 '40231795',
 '461771385',
 '1420248766423126017',
 '181425795',
 '1402438217962508296',
 '298190124',
 '68569193',
 '117978497',
 '291906276',
 '893899187741261824',
 '878024999382245376',
 '1236741840629649414',
 '29651030',
 '47789589',
 '33888973',
 '33